In [14]:
!pip install flash-attn --no-build-isolation

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [15]:
!pip install git+https://github.com/TIGER-AI-Lab/Mantis.git

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  Cloning https://github.com/TIGER-AI-Lab/Mantis.git to /tmp/pip-req-build-31xuqyvj
  Running command git clone --filter=blob:none --quiet https://github.com/TIGER-AI-Lab/Mantis.git /tmp/pip-req-build-31xuqyvj
  Resolved https://github.com/TIGER-AI-Lab/Mantis.git to commit fa5a57bd07a32558ff95356c7f76c0a926b4bb4d
  Running command git submodule update --init --recursive -q
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [ ]:
from mantis.models.mllava import chat_mllava
from PIL import Image
import flash_attn_2_cuda
import torch


In [10]:
torch.cuda.empty_cache()

In [ ]:
image1 = "image1.png"
image2 = "image2.png"
images = [Image.open(image1), Image.open(image2)]

# load processor and model
from mantis.models.mllava import MLlavaProcessor, LlavaForConditionalGeneration
processor = MLlavaProcessor.from_pretrained("TIGER-Lab/Mantis-8B-siglip-llama3")
attn_implementation = None #flash_attn_2_cuda
model = LlavaForConditionalGeneration.from_pretrained("TIGER-Lab/Mantis-8B-siglip-llama3", device_map="auto", torch_dtype=torch.bfloat16, attn_implementation=attn_implementation)

generation_kwargs = {
    "max_new_tokens": 24,
    "num_beams": 1,
    "do_sample": False
}

# chat
text = "Describe the difference of <image> and <image> as much as you can."
response, history = chat_mllava(text, images, model, processor, **generation_kwargs)

print("USER: ", text)
print("ASSISTANT: ", response)

text = "How many wallets are there in image 1 and image 2 respectively?"
response, history = chat_mllava(text, images, model, processor, history=history, **generation_kwargs)

print("USER: ", text)
print("ASSISTANT: ", response)


In [ ]:
# Load JSON file
input_file = "/home/g2/ChartQA/ChartQA Dataset/test/test_human.json"  # Replace with your input JSON file path
output_file = "output.json"  # File to save generated answers

with open(input_file, "r") as f:
    data = json.load(f)

In [ ]:
processor = Pix2StructProcessor.from_pretrained('google/deplot')
model = Pix2StructForConditionalGeneration.from_pretrained('google/deplot')

url = "https://raw.githubusercontent.com/vis-nlp/ChartQA/main/ChartQA%20Dataset/val/png/5090.png"
image = Image.open(requests.get(url, stream=True).raw)

inputs = processor(images=image, text="Generate underlying data table of the figure below:", return_tensors="pt")
predictions = model.generate(**inputs, max_new_tokens=512)
print(processor.decode(predictions[0], skip_special_tokens=True))

In [ ]:
# Initialize model
processor = Pix2StructProcessor.from_pretrained('google/deplot')
model = Pix2StructForConditionalGeneration.from_pretrained('google/deplot')

# List to store results
results = []

previous_file_name = ""

# Process each entry in the JSON file
for entry in data:
    file_name = entry["imgname"]
    file_path = "/home/g2/ChartQA/ChartQA Dataset/test/png/{}".format(file_name)
    question = entry["query"]
    expected_answer = entry["label"]

    print(type(question))

    # Reload pipeline if the file name changes
    if file_name != previous_file_name:
        print(f"Loading new image: {file_name}")
        processor = Pix2StructProcessor.from_pretrained('google/deplot')
        model = Pix2StructForConditionalGeneration.from_pretrained('google/deplot')
        previous_file_name = file_name  # Update tracking variable

    # Open the image and verify it's valid
    try:
        image = Image.open(file_path)
        width, height = image.size
        # Validate image dimensions (width and height must not be None)
        if width is None or height is None:
            print(f"Invalid image dimensions for {file_path}")
            continue  # Skip this image if dimensions are invalid
    except (IOError, SyntaxError) as e:
        print(f"Error loading image {file_path}: {e}")
        continue  # Skip this image and move to the next one

    inputs = processor(images=image, text="Using the chart provided, answer the question: {}".format(question), return_tensors="pt")
    predictions = model.generate(**inputs, max_new_tokens=512)
    generated_answer = processor.decode(predictions[0], skip_special_tokens=True)
    print(question)
    print(generated_answer)

    # Store the result
    results.append({
        "imgname": file_name,
        "query": question,
        "label": expected_answer,
        "generated_answer": generated_answer
    })

# Save results to output JSON
with open(output_file, "w") as f:
    json.dump(results, f, indent=4)

print(f"Results saved to {output_file}")